<div class="alert alert-block alert-info">

# Index
    
[1. Install libraries](#1)<br>
    
[2. Import libraries](#2)<br>
    
[3. Check File Statistics](#3)<br>
  
[4. Deal with Missing Values](#4)<br>
    
[5. Check and Fix Inconsistencies](#5)<br>

[6. Check and Delete Outliers](#6)<br>
    
[7. Feature Engeneering](#7)<br>

[8. Feature Selection](#8)<br>

[9. Data Normalization](#9)<br>
    
- [9.1 Min Max Scaler](#9.1)<br>
    
- [9.2 One Hot Encoding](#9.2)<br>  

[10. Using DBScan to remove outliers](#10)<br>

[11. Dimensionality Reduction: PCA](#11)<br>
    
[12. Models](#12)<br>
    
- [12.1 Hierarchical Clustering](#12.1)<br>
    
- [12.2 K Means Clustering](#12.2)<br>  

- [12.3 Self-Organizing Maps](#12.3)<br>
    - [12.3.1 K Means on top of SOM](#12.3.1)<br>  
    - [12.3.2 Hierarchical Clustering on top of SOM](#12.3.2)<br>

- [12.4 Birch Clustering](#12.4)<br>
  
- [12.5 Clustering by Perspectives](#12.5)<br>
    - [12.5.1 K Means Clustering](#12.5.1)<br>  
    - [12.5.2 Merging Perspectives](#12.5.2)<br>
    
[13. Cluster Visualization](#13)<br>
    
- [13.1 Using t-SNE](#13.1)<br>  
    
- [13.2 Using Polar Line Plot](#13.2)<br>
    
- [13.3 Using Cluster Profiles](#13.3)<br>

    
</div>

<a class="anchor" id="1">

# 1. Install libraries
    
</a>

In [ ]:
!pip install sas7bdat

In [ ]:
!pip install SOMPY-master.zip

In [ ]:
!pip install ipdb

In [ ]:
!pip install ipdb==0.8.1

In [ ]:
!pip install graphviz

In [ ]:
!pip install ipywidgets

<a class="anchor" id="2">

# 2. Import libraries
    
</a>

In [ ]:
import sqlite3
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from itertools import product
from pandas_profiling import ProfileReport
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder
#For PCA
from sklearn.decomposition import PCA
import plotly.express as px
#For Hierarcical clustering
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.cm as cm
from sklearn.cluster import AffinityPropagation
from numpy import unique
from numpy import where
from sklearn.cluster import Birch as Birch
from sklearn.neighbors import NearestNeighbors
from sklearn.mixture import GaussianMixture
from sklearn.cluster import MeanShift, DBSCAN, estimate_bandwidth
from sklearn.cluster import SpectralClustering
#------ SOM-------
from os.path import join
from sklearn.neighbors import KNeighborsClassifier
import joblib
import sompy
from sompy.visualization.mapview import View2D
from sompy.visualization.bmuhits import BmuHitsView
from sompy.visualization.hitmap import HitMapView
from matplotlib.patches import RegularPolygon, Ellipse
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm, colorbar
from matplotlib import colors as mpl_colors
from matplotlib.lines import Line2D
from matplotlib import __version__ as mplver
#----- kmean-----
import seaborn as sns
import matplotlib.cm as cm
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import KMeans
#-----hierarchical clustering----
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

from collections import Counter
from os.path import join
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.base import clone
from sklearn.metrics import pairwise_distances
from scipy.cluster.hierarchy import dendrogram
from sklearn.manifold import TSNE
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
import graphviz
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [ ]:
from sas7bdat import SAS7BDAT
with SAS7BDAT(r'a2z_insurance.sas7bdat') as file:
    df = file.to_data_frame()

<a class="anchor" id="3">

# 3. Check File Statistics
    
</a>

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df

In [ ]:
# Replacing missing values with numpy NaN
df.replace("", np.nan, inplace=True)

In [ ]:
df.describe(include="all").T

In [ ]:
# Separating metric features from non metric features
non_metric_features = ["EducDeg", "GeoLivArea","Children"]
metric_features = df.columns.drop(non_metric_features).to_list()

In [ ]:
# Checking for missing values
df_central = df.copy()
df_central.isna().sum()

<a class="anchor" id="4">

# 4. Deal with Missing Values
    
</a>

Filling missing values with mode for non-metric features and median for metric features

In [ ]:
modes = df_central[non_metric_features].mode().loc[0]
modes

In [ ]:
medians = df_central[metric_features].median()
medians

In [ ]:
imp_values = pd.concat([medians,modes]) 

In [ ]:
df_central.fillna(imp_values, inplace=True)
df_central.isna().sum()  

<a class="anchor" id="5">

# 5. Check and Fix Inconsistencies
    
</a>

In [ ]:
df_central[df_central.BirthYear < df_central.FirstPolYear]

#There is inconsistency, however we decided to ignore it as later the birthYear will be removed

<a class="anchor" id="6">

# 6. Check and Delete Outliers
    
</a>

In [ ]:
sns.set()
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(20, 11))
for ax, feat in zip(axes.flatten(), metric_features): 
    sns.boxplot(x=df_central[feat], ax=ax)
title = "Figure 1: Numeric Variables' Box Plots with outliers"
plt.suptitle(title, size=30)
plt.savefig('outliers1m.png')
plt.show()

The dataset contain many outliers, as it seen in FirstPolYear, BirthYear or PremHealth

In [ ]:
df_original = df.copy()

In [ ]:
#Manual filtering method
filters1 = (
    (df_central['FirstPolYear']<=2016)
    &
    (df_central['BirthYear']>=1916)
    &
    (df_central['ClaimsRate']<=100)
    &
    (df_central['MonthSal']<=20000)
    &
    (df_central['PremMotor']<=2000)
    &
    (df_central['PremHousehold']<=1400)
    &
    (df_central['PremHealth']<=5000)
    &
    (df_central['PremWork']<=250)
    &
    (df_central['CustMonVal']>=-2000)
    &
    (df_central['ClaimsRate']<=2)
    &
    (df_central['CustMonVal']<=1500)
    &
    (df_central['PremLife']<=250)
    
)

df_1 = df_central[filters1]

print('Percentage of data kept after removing outliers:', np.round(df_1.shape[0] / df_original.shape[0], 4))

In [ ]:
# Quartile method

q25 = df_central.quantile(.25)
q75 = df_central.quantile(.75)
iqr = (q75 - q25)

upper_lim = q75 + 1.5 * iqr
lower_lim = q25 - 1.5 * iqr

filters2 = []
for metric in metric_features:
    llim = lower_lim[metric]
    ulim = upper_lim[metric]
    filters2.append(df_central[metric].between(llim, ulim, inclusive='both'))

filters2 = pd.Series(np.all(filters2, 0))
df_2 = df_central[filters2]
print('Percentage of data kept after removing outliers:', np.round(df_2.shape[0] / df_original.shape[0], 4))

In [ ]:
# Mixed method (quartile + filter)

df_3 = df_central[(filters1 | filters2)]

print('Percentage of data kept after removing outliers:', np.round(df_3.shape[0] / df_original.shape[0], 4))

In [ ]:
df_central = df_3.copy()

In [ ]:
sns.set()
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(20, 11))
for ax, feat in zip(axes.flatten(), metric_features): 
    sns.boxplot(x=df_central[feat], ax=ax)
title = "Figure 2: Numeric Variables' Box Plots"
plt.suptitle(title, size=30)
plt.savefig('outliers2m.png')
plt.show()

The dataset now look more balanced, as many less records are placed outise the quartile range

In [ ]:
sns.set()
fig, axes = plt.subplots(2, ceil(len(non_metric_features) / 2), figsize=(20, 11))
for ax, feat in zip(axes.flatten(), non_metric_features): 
    ax.hist(df_central[feat])
    ax.set_title(feat)
title = "Non Metric Variables' Histograms"
plt.suptitle(title)
plt.savefig('outliers1nm.png')
plt.show()

The non_metric features do not containing any visible outlier


<a class="anchor" id="7">

# 7. Feature Engeneering
    
</a>

In [ ]:
# Create a new features called birthyear based on age
df_central['Age'] = 2016 - df_central['BirthYear']
metric_features.append('Age')

In [ ]:
# Create a new features for the number of years since first policy happen
df_central['PolicyYears'] = 2016 - df_central['FirstPolYear']
metric_features.append('PolicyYears')

In [ ]:
#Create a new feature resulting from the sum of all premiums
df_central['TotalPremiums'] = df_central['PremLife'] + df_central['PremHousehold'] + df_central['PremHealth'] + df_central['PremWork'] + df_central['PremMotor']
metric_features.append('TotalPremiums')

In [ ]:
#Create a new features called yearly salary based on the month salary
df_central['YearSal'] = df_central['MonthSal']*12
metric_features.append('YearSal')

In [ ]:
#Create a new features to represent what percentage of the yearly salary is spent on premiums
df_central['Premiums%Salary'] = df_central['TotalPremiums'] / df_central['YearSal']
metric_features.append('Premiums%Salary')

<a class="anchor" id="8">

# 8. Feature Selection
    
</a>

In [ ]:
fig = plt.figure(figsize=(10, 8))
corr = np.round(df_central[metric_features].corr(method="pearson"), decimals=2)
mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape,""))
sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)
fig.subplots_adjust(top=0.95)
fig.suptitle("Figure 3: Correlation Matrix", fontsize=20)
plt.savefig('CorMtx1.png')
plt.show()

In [ ]:
#Remove BirthYear and MonthSal features
df_central.drop('BirthYear', inplace=True, axis=1)
metric_features.remove('BirthYear')
df_central.drop('MonthSal', inplace=True, axis=1)
metric_features.remove('MonthSal')

In [ ]:
#Remove Age feature
df_central.drop('Age',inplace=True,axis=1)
metric_features.remove('Age')

In [ ]:
#Remove PolicyYear and FirstPolYear features
df_central.drop('PolicyYears',inplace=True,axis=1)
metric_features.remove('PolicyYears')
df_central.drop('FirstPolYear',inplace=True,axis=1)
metric_features.remove('FirstPolYear')

In [ ]:
#Remove CustID feature
df_central.drop('CustID',inplace=True,axis=1)
metric_features.remove('CustID')

In [ ]:
#Remove ClaimsRate feature
df_central.drop('ClaimsRate',inplace=True,axis=1)
metric_features.remove('ClaimsRate')

In [ ]:
#Correlation Matrix after the features selection
fig = plt.figure(figsize=(10, 8))

corr = np.round(df_central[metric_features].corr(method="pearson"), decimals=2)
mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape,"")) 
sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)
fig.subplots_adjust(top=0.95)
fig.suptitle("Correlation Matrix", fontsize=20)

plt.show()

<a class="anchor" id="9">

# 9. Data Normalization
    
</a>

<a class="anchor" id="9.1">

## 9.1. Min Max Scaler
    
</a>

In [ ]:
df_minmax = df_central.copy()

In [ ]:
# Use MinMaxScaler to scale the data
scaler = MinMaxScaler()
scaled_feat = scaler.fit_transform(df_minmax[metric_features])
scaled_feat

In [ ]:
df_minmax[metric_features] = scaled_feat
df_minmax.head()

In [ ]:
df_minmax[metric_features].describe().round(2)

<a class="anchor" id="9.2">

## 9.2. One Hot Encoding
    
</a>

In [ ]:
df_ohc = df_minmax.copy()

In [ ]:
# Use OneHotEncoder to encode the categorical features. Get feature names and create a DataFrame 

ohc = OneHotEncoder(sparse=False, drop="first")
ohc_feat = ohc.fit_transform(df_ohc[non_metric_features])
ohc_feat_names = ohc.get_feature_names_out()
ohc_df = pd.DataFrame(ohc_feat, index=df_ohc.index, columns=ohc_feat_names)
ohc_df

In [ ]:
# Reassigning df to contain ohc variables
df_ohc = pd.concat([df_ohc.drop(columns=non_metric_features), ohc_df], axis=1)
df_ohc.head()

<a class="anchor" id="10">

# 10. Using DBScan to remove outliers
    
</a>

In [ ]:
#DBSCAN method to check for outliers
dbscan = DBSCAN(eps=0.3, min_samples=30, n_jobs=4)
dbscan_labels = dbscan.fit_predict(df_ohc[metric_features])
Counter(dbscan_labels)

In [ ]:
# Save outliers in a dataframe to analyse them later
df_outliers_dbscan = df_ohc[dbscan_labels==-1].copy()

In [ ]:
# New df without outliers
df_before_models = df_ohc[dbscan_labels!=-1].copy()

In [ ]:
# Check Box Plots after removing the outliers with dbscan
sns.set()
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(20, 11))
for ax, feat in zip(axes.flatten(), metric_features): 
    sns.boxplot(x=df_before_models[feat], ax=ax)         
title = "Numeric Variables' Box Plots"
plt.suptitle(title)
plt.show()

<a class="anchor" id="11">

# 11. Dimensionality Reduction: PCA
    
</a>

In [ ]:
df_pca=df_before_models.copy()

In [ ]:
df_pca[metric_features]

In [ ]:
#STEP 1: STANDARDIZATION - already done with the normalization with Min Max Scaler to the continuous variables [metric_]
#STEP 2: COVARIANCE MATRIX COMPUTATION

pca = PCA()
pca_feat =  pca.fit_transform(df_pca[metric_features])
pca_feat.shape  # What is this output?

In [ ]:
# Output PCA table
# STEP 3: COMPUTE THE EIGENVECTORS AND EIGENVALUES OF THE COVARIANCE MATRIX TO IDENTIFY THE PRINCIPAL COMPONENTS
pd.DataFrame(
    {"Eigenvalue": pca.explained_variance_,
     "Difference": np.insert(np.diff(pca.explained_variance_), 0, 0),
     "Proportion": pca.explained_variance_ratio_,
     "Cumulative": np.cumsum(pca.explained_variance_ratio_)},
    index=range(1, pca.n_components_ + 1)
)

In [ ]:
# figure and axes
fig,(ax1, ax2) = plt.subplots(1,2, figsize=(15,5))

# draw plots
ax1.plot(pca.explained_variance_, marker=".", markersize=12)  # CODE HERE: PLOT THE EIGENVALUES (EXPLAINED VARIANCE)
ax2.plot(pca.explained_variance_ratio_, marker=".", markersize=12, label="Proportion")  # CODE HERE: PLOT THE EXPLAINED VARIANCE RATIO
ax2.plot(np.cumsum(pca.explained_variance_ratio_), marker=".", markersize=12, linestyle="--", label="Cumulative")  # CODE HERE: PLOT THE CUMULATIVE EXPLAINED VARIANCE RATIO

# customizations
ax2.legend()
ax1.set_title("Scree Plot", fontsize=14)
ax2.set_title("Variance Explained", fontsize=14)
ax1.set_ylabel("Eigenvalue")
ax2.set_ylabel("Proportion")
ax1.set_xlabel("Components")
ax2.set_xlabel("Components")
ax1.set_xticks(range(0, pca.n_components_, 2))
ax1.set_xticklabels(range(1, pca.n_components_ + 1, 2))
ax2.set_xticks(range(0, pca.n_components_, 2))
ax2.set_xticklabels(range(1, pca.n_components_ + 1, 2))

plt.show()

From these graphics we see that: the 1st PC is very important with almost 50% of all data; and with 5 components we only loose around 6% of the data. Let´s try then with 6.

In [ ]:
# Perform PCA again with the number of principal components you want to retain
# STEP 4: FEATURE VECTOR
pca = PCA(n_components=6)
pca_feat = pca.fit_transform(df_pca[metric_features])
pca_feat_names = [f"PC{i}" for i in range(pca.n_components_)]
pca_df = pd.DataFrame(pca_feat, index=df_pca.index, columns=pca_feat_names)  # remember index=df_pca.index
pca_df

In [ ]:
# Reassigning df to contain pca variables
# LAST STEP: RECAST THE DATA ALONG THE PRINCIPAL COMPONENTS AXES
df_pca = pd.concat([df_pca, pca_df], axis=1)
df_pca.head()

Let´s interpret each Principal Component (with style)

In [ ]:
def _color_red_or_green(val):
    if val < -0.45:
        color = 'background-color: red'
    elif val > 0.45:
        color = 'background-color: green'
    else:
        color = ''
    return color

# Interpreting each Principal Component
# CODE HERE: Obtain the loadings (i.e. correlation between PCs and original features)
loadings = df_pca[metric_features + pca_feat_names].corr().loc[metric_features, pca_feat_names]
loadings.style.applymap(_color_red_or_green)

From this colorful table we can conclude that we can eliminate PC4 and PC5 as their highest  correlations are already taking in a count in PC0 and PC3, respectively.

In [ ]:
df_pca.drop(columns=['PC4','PC5'], inplace=True)

In [ ]:
df_pca

In [ ]:
ProfileReport(
    df_pca,
    title='A2Z Insurance Data Preprocessed',
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
    },
)

Now just plot a graphic between PC0 and PC1:

In [ ]:
#Visualizing in 2D for PC0 and PC1:
fig = px.scatter(df_pca, x="PC0", y="PC1")
fig.show()

In [ ]:
#3D:
fig = px.scatter_3d(
    df_pca, x="PC0", y="PC1", z="PC2",
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

In [ ]:
features = ["PC0", "PC1","PC2","PC3"]

fig = px.scatter_matrix(
    df_pca,
    dimensions=features
)
fig.update_traces(diagonal_visible=False)
fig.show()

<a class="anchor" id="12">

# 12. Models
    
</a>

<a class="anchor" id="12.1">

## 12.1. Hierarchical Clustering
    
</a>

In [ ]:
df_hc = df_before_models.copy()

Defining the linkage method to choose:

In [ ]:
def get_r2_hc(df, link_method, max_nclus, min_nclus=1, dist="euclidean"):
    """This function computes the R2 for a set of cluster solutions given by the application of a hierarchical method.
    The R2 is a measure of the homogenity of a cluster solution. It is based on SSt = SSw + SSb and R2 = SSb/SSt. 
    
    Parameters:
    df (DataFrame): Dataset to apply clustering
    link_method (str): either "ward", "complete", "average", "single"
    max_nclus (int): maximum number of clusters to compare the methods
    min_nclus (int): minimum number of clusters to compare the methods. Defaults to 1.
    dist (str): distance to use to compute the clustering solution. Must be a valid distance. Defaults to "euclidean".
    
    Returns:
    ndarray: R2 values for the range of cluster solutions
    """
    def get_ss(df):
        ss = np.sum(df.var() * (df.count() - 1))
        return ss  # return sum of sum of squares of each df variable
    sst = get_ss(df)  # get total sum of squares
    r2 = []  # where we will store the R2 metrics for each cluster solution
    for i in range(min_nclus, max_nclus+1):  # iterate over desired ncluster range
        cluster = AgglomerativeClustering(n_clusters=i, affinity=dist, linkage=link_method)
        hclabels = cluster.fit_predict(df) #get cluster labels
        df_concat = pd.concat((df, pd.Series(hclabels, name='labels')), axis=1)  # concat df with labels
        ssw_labels = df_concat.groupby(by='labels').apply(get_ss)  # compute ssw for each cluster labels
        ssb = sst - np.sum(ssw_labels)  # remember: SST = SSW + SSB
        r2.append(ssb / sst)  # save the R2 of the given cluster solution
    return np.array(r2)

In [ ]:
def draw_r2_plot_hierarchical(df_hc,hc_methods,distance):
    # Call function defined above to obtain the R2 statistic for each hc_method
    max_nclus = 10
    r2_hc_methods = np.vstack(
        [
            get_r2_hc(df=df_hc[metric_features], link_method=link, max_nclus=max_nclus,dist = distance) 
            for link in hc_methods
        ]
    ).T
    r2_hc_methods = pd.DataFrame(r2_hc_methods, index=range(1, max_nclus + 1), columns=hc_methods)

    sns.set()
    # Plot data
    fig = plt.figure(figsize=(11,5))
    sns.lineplot(data=r2_hc_methods, linewidth=2.5, markers=["o"]*len(hc_methods))

    # Finalize the plot
    fig.suptitle("R2 plot for various hierarchical methods", fontsize=21)
    plt.gca().invert_xaxis()  # invert x axis
    plt.legend(title="HC methods", title_fontsize=11)
    plt.xticks(range(1, max_nclus + 1))
    plt.xlabel("Number of clusters", fontsize=13)
    plt.ylabel("R2 metric", fontsize=13)

    plt.show()

In [ ]:
methods = ["ward", "complete", "average", "single"]
distance = 'euclidean'
draw_r2_plot_hierarchical(df_hc,methods,distance)

For the Euclidean distance: We conclude that from 1 to 5 clusters, the best method is the Complete one

Then, for 6 clusters is the Single one

And after that is the Ward one

Now let´s test for the Manhattan distance:

In [ ]:
methods = ["complete", "average", "single"]
distance = 'manhattan'
draw_r2_plot_hierarchical(df_hc,methods,distance)

The Manhattan distance is not as good as the Euclidean. So let´s keep the euclidean one

Defining the number of clusters:

In [ ]:
linkage = 'complete'
distance = 'euclidean'
hclust = AgglomerativeClustering(linkage=linkage, affinity=distance, distance_threshold=0, n_clusters=None)
hclust.fit_predict(df_hc[metric_features])

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

def draw_dendrogram(y_threshold,hclust):
    # create the counts of samples under each node (number of points being merged)
    counts = np.zeros(hclust.children_.shape[0])
    n_samples = len(hclust.labels_)

    # hclust.children_ contains the observation ids that are being merged together
    # At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
    for i, merge in enumerate(hclust.children_):
        # track the number of observations in the current cluster being formed
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                # If this is True, then we are merging an observation
                current_count += 1  # leaf node
            else:
                # Otherwise, we are merging a previously formed cluster
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    # the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
    # the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
    # the counts indicate the number of points being merged (dendrogram's x-axis)
    linkage_matrix = np.column_stack(
        [hclust.children_, hclust.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    sns.set()
    fig = plt.figure(figsize=(11,5))
    # The Dendrogram parameters need to be tuned
    dendrogram(linkage_matrix, truncate_mode='level', p=5, color_threshold=y_threshold, above_threshold_color='k')
    plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
    plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram', fontsize=21)
    plt.xlabel('Number of points in node (or index of point if no parenthesis)')
    plt.ylabel(f'{distance.title()} Distance', fontsize=13)
    plt.show()

In [ ]:
draw_dendrogram(1.5,hclust)

Final Hierarchical clustering solution:

In [ ]:
# 4 cluster solution
linkage = 'complete'
distance = 'euclidean'
hc4lust = AgglomerativeClustering(linkage=linkage, affinity=distance, n_clusters=4)
hc4_labels = hc4lust.fit_predict(df_hc[metric_features])

In [ ]:
# Characterizing the 4 clusters
df_final_hc = pd.concat((df_hc, pd.Series(hc4_labels, name='labels')), axis=1)
df_final_hc.groupby('labels').mean()

In [ ]:
# using R²
def get_ss(df):
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable

def get_r2(df):
    sst = get_ss(df[metric_features])  # get total sum of squares
    ssw_labels = df[metric_features + ["labels"]].groupby(by='labels').apply(get_ss)  # compute ssw for each cluster labels
    ssb = sst - np.sum(ssw_labels)  # remember: SST = SSW + SSB
    r2 = ssb / sst
    return r2

In [ ]:
get_r2(df_final_hc)

<a class="anchor" id="12.2">

## 12.2. K Means Clustering
    
</a>

In [ ]:
df_kmeans = df_before_models.copy()

Defining the number of clusters:

In [ ]:
r_clusters = range(2, 11)

In [ ]:
inertia = []
for n_clus in r_clusters: 
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=15, random_state=1)
    kmclust.fit(df_kmeans[metric_features])
    inertia.append(kmclust.inertia_)  

In [ ]:
def draw_inertia_plot(range_clusters,inertia_values):
    # The inertia plot
    plt.figure(figsize=(9,5))
    plt.plot(range_clusters,inertia_values)
    plt.ylabel("Inertia: SSw")
    plt.xlabel("Number of clusters")
    plt.title("Inertia plot over clusters", size=15)
    plt.show()

In [ ]:
draw_inertia_plot(r_clusters,inertia)

From this graphic and using the elbow method, the best nº of clusters is 4

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

def draw_silhouette_plot(range_clusters,df):
    avg_silhouette = []
    for nclus in range_clusters:
        # Skip nclus == 1
        if nclus == 1:
            continue
        # Create a figure
        fig = plt.figure(figsize=(13, 7))
        # Initialize the KMeans object with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        kmclust = KMeans(n_clusters=nclus, init='k-means++', n_init=15, random_state=1)
        cluster_labels = kmclust.fit_predict(df[metric_features])
        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed clusters
        silhouette_avg = silhouette_score(df[metric_features], cluster_labels)
        avg_silhouette.append(silhouette_avg)
        print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")
        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(df[metric_features], cluster_labels)
        y_lower = 10
        for i in range(nclus):
            # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
            ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
            ith_cluster_silhouette_values.sort()
            # Get y_upper to demarcate silhouette y range size
            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i
            # Filling the silhouette
            color = cm.nipy_spectral(float(i) / nclus)
            plt.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples


        plt.title("The silhouette plot for the various clusters.")
        plt.xlabel("The silhouette coefficient values")
        plt.ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        plt.axvline(x=silhouette_avg, color="red", linestyle="--")

        # The silhouette coefficient can range from -1, 1
        xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
        plt.xlim([xmin, xmax])
        plt.ylim([0, len(df[metric_features]) + (nclus + 1) * 10])
        plt.yticks([]) 
        plt.xticks(np.arange(xmin, xmax, 0.1))
    return avg_silhouette

In [ ]:
average_silhouette = draw_silhouette_plot(r_clusters,df_kmeans)

In [ ]:
###### AVERAGE SILHOUETTE PLOT #############

def draw_average_silhouette(range_clusters,avg_silhouette):
    plt.figure(figsize=(9,5))
    plt.plot(range_clusters,avg_silhouette)
    plt.ylabel("Average silhouette")
    plt.xlabel("Number of clusters")
    plt.title("Average silhouette plot over clusters", size=15)
    plt.show()

In [ ]:
draw_average_silhouette(r_clusters,average_silhouette)

After seeing this graphic, since the average silhouette for 3 and 4 clusters is very similar the best choice might be 3.

In [ ]:
# final cluster solution
number_clusters = 3
kmclust = KMeans(n_clusters=number_clusters, init='k-means++', n_init=15, random_state=1)
km_labels = kmclust.fit_predict(df_kmeans[metric_features])
km_labels

In [ ]:
# Characterizing the final clusters
df_final_kmeans = pd.concat((df_kmeans, pd.Series(km_labels, name='labels')), axis=1)
df_final_kmeans.groupby('labels').mean()

In [ ]:
get_r2(df_final_kmeans)

Final R square value for K means is 0.0373747

<a class="anchor" id="12.3">

## 12.3. Self-Organizing Maps
    
</a>


In [ ]:
df_som = df_before_models.copy()

In [ ]:
#analyse initially with a lower mapsize of 10x10
np.random.seed(42)

sm = sompy.SOMFactory().build(
    df_som[metric_features].values, 
    mapsize=[10, 10], 
    initialization='random', 
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=metric_features
)
sm.train(n_job=4, verbose='info', train_rough_len=100, train_finetune_len=100)

In [ ]:
plt.rcParams['figure.dpi'] = 72
weights = sm.codebook.matrix 

In [ ]:
###################################
### Visualizing Component Planes ##
###################################

def plot_component_planes(weights,
                          features,
                          M=3, N=4, 
                          figsize=(20,20),
                          figlayout=(3,4),
                          title="Component Planes",
                          cmap=cm.magma
                         ):
    
    xx, yy = np.meshgrid(np.arange(M), np.arange(N))
    xx = xx.astype(float)
    yy = yy.astype(float)
    xx[::-2] -= 0.5
    xx = xx.T
    yy = yy.T
    weights_ = weights.reshape((M,N,len(features)))
    fig = plt.figure(figsize=figsize, constrained_layout=True)
    subfigs = fig.subfigures(figlayout[0], figlayout[1], wspace=.15)
    ## Normalize color scale to range of all values
    colornorm = mpl_colors.Normalize(vmin=np.min(weights), 
                                         vmax=np.max(weights))
    for cpi, sf in zip(range(len(metric_features)), subfigs.flatten()):

        sf.suptitle(features[cpi], y=0.95)
        axs = sf.subplots(1,1, )
        axs.set_aspect('equal')

        ## Normalize color scale to range of values in each component
        colornorm = mpl_colors.Normalize(vmin=np.min(weights_[:,:,cpi]), 
                                         vmax=np.max(weights_[:,:,cpi]))

        # iteratively add hexagons
        for i in range(weights_.shape[0]):
            for j in range(weights_.shape[1]):
                wy = yy[(i, j)] * np.sqrt(3) / 2
                hexagon = RegularPolygon((xx[(i, j)], wy), 
                                     numVertices=6, 
                                     radius=.99 / np.sqrt(3),
                                     facecolor=cmap(colornorm(weights_[i, j, cpi])), 
                                     alpha=1, 
                                     linewidth=.5,
                                     edgecolor=cmap(colornorm(weights_[i, j, cpi]))
                                    )
                axs.add_patch(hexagon)

        ## only run this block if matplotlib >= 3.6.x
        mplv = [int(i) for i in mplver.split('.')]
        if mplv[1] >= 6:
            ## Add colorbar
            divider = make_axes_locatable(axs)
            ax_cb = divider.append_axes("right", size="7%")#, pad="2%")

            ## Create a Mappable object
            cmap_sm = plt.cm.ScalarMappable(cmap=cmap, norm=colornorm)
            cmap_sm.set_array([])

            ## Create custom colorbar 
            cb1 = colorbar.Colorbar(ax_cb,
                                    orientation='vertical', 
                                    alpha=1,
                                    mappable=cmap_sm
                                    )
            cb1.ax.get_yaxis().labelpad = 16

            ## Add colorbar to plot
            sf.add_axes(ax_cb)

        ## Remove axes for hex plot
        axs.margins(.05)
        axs.axis("off")
    fig.suptitle(title)
    plt.show()

In [ ]:
print("matplotlib version is:" , mplver)

## Component Planes

In [ ]:
# ## Run this cell ONLY if matplotlib version is <= 3.4.x
# ## Comment out this cell and run the cell below instead if it doesn't work.


# # Visualizing the Component planes (feature values)
sns.set()
view2D = View2D(12, 12, "", text_size=10)
view2D.show(sm, col_sz=3, what='codebook')
plt.subplots_adjust(top=0.90)
plt.suptitle("Component Planes", fontsize=20)
plt.show()

In [ ]:
# ## Run this cell ONLY if matplotlib version is > 3.4.x
#plot_component_planes(weights=sm.codebook.matrix,features=metric_features,
#                     M=10,N=10,
#                     figsize=(12,15),figlayout=(4,3),
#                     title="Component Planes",
#                     cmap=sns.color_palette("rocket", as_cmap=True))

## U-matrix

In [ ]:
# Here you have U-matrix
u = sompy.umatrix.UMatrixView(9, 9, 'umatrix', show_axis=True, text_size=8, show_text=True)

UMAT = u.show(
    sm, 
    distance=2,
    row_normalized=False,
    show_data=True, 
    contour=True, # Visualize isomorphic curves
    blob=True
)

np.flip(UMAT[1], axis=1) # U-matrix values - they match with the plot colors

## Hit-map

In [ ]:
vhts  = BmuHitsView(12,12,"Hits Map")
vhts.show(sm, anotate=True, onlyzeros=False, labelsize=12, cmap="Blues")
plt.show()

## Unfolding phase
### Clustering with SOMs: K-means SOM vs Emergent SOM

In [ ]:
#Repeat the analisy with an higher map size of 50 x 50
np.random.seed(42)

sm = sompy.SOMFactory().build(
    df_som[metric_features].values, 
    mapsize=[50, 50],  
    initialization='random',
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=metric_features
)
sm.train(n_job=-1, verbose='info', train_rough_len=100, train_finetune_len=100)

In [ ]:
########### MATPLOTLIB VERSON <= 3.4 ##############
sns.set()
view2D = View2D(12, 12, "", text_size=10)
view2D.show(sm, col_sz=3, what='codebook')
plt.subplots_adjust(top=0.90)
plt.suptitle("Component Planes", fontsize=20)
plt.show()

In [ ]:
#plot_component_planes(weights=sm.codebook.matrix,features=metric_features,
#                     M=50,N=50, # change to 50 50 if chnaged in codeline before
#                     figsize=(12,15),figlayout=(4,3),
#                     title="Component Planes 50x50",
#                     cmap=sns.color_palette("rocket", as_cmap=True))

In [ ]:
# U-matrix of the 50x50 grid
u = sompy.umatrix.UMatrixView(9, 9, 'umatrix', show_axis=True, text_size=8, show_text=True)

UMAT = u.show(
    sm, 
    distance=2, 
    row_normalized=False, 
    show_data=False, 
    contour=True # Visualize isomorphic curves
)

<a class="anchor" id="12.3.1">

### 12.3.1. K Means on top of SOM
    
</a>


In [ ]:
df_somK = df_som.copy()

In [ ]:
r_clusters = range(2, 11)
inertia = []
for n_clus in r_clusters:  
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=15, random_state=1)
    kmclust.fit(df_somK[metric_features])
    inertia.append(kmclust.inertia_) 

In [ ]:
draw_inertia_plot(r_clusters,inertia)

The number of cluster fo k-Means is 4

In [ ]:
average_s = draw_silhouette_plot(r_clusters,df_somK)

In [ ]:
draw_average_silhouette(r_clusters,average_s)

we select 3 clusters as the difference with 4 is very low

In [ ]:
# Perform K-Means clustering on top of the 2500 untis (sm.get_node_vectors() output)
kmeans = KMeans(n_clusters=3, init='k-means++', n_init=20, random_state=42)
nodeclus_labels = kmeans.fit_predict(sm.codebook.matrix)
sm.cluster_labels = nodeclus_labels  # setting the cluster labels of sompy

hits = HitMapView(12, 12,"Clustering", text_size=10)
hits.show(sm, anotate=True, onlyzeros=False, labelsize=7, cmap="Pastel1")

plt.show()

In [ ]:
# Check the nodes and and respective clusters
nodes = sm.codebook.matrix

df_nodes = pd.DataFrame(nodes, columns=metric_features)
df_nodes['labels'] = nodeclus_labels
df_nodes

In [ ]:
# Obtaining SOM's BMUs labels
bmus_map = sm.find_bmu(df_somK[metric_features])[0]  # get bmus for each observation in df

df_bmus = pd.DataFrame(
    np.concatenate((df_somK, np.expand_dims(bmus_map,1)), axis=1),
    index=df_somK.index, columns=np.append(df_somK.columns,"BMU")
)
df_bmus

In [ ]:
# Get cluster labels for each observation
df_final_somK = df_bmus.merge(df_nodes['labels'], 'left', left_on="BMU", right_index=True)
df_final_somK

In [ ]:
# Characterizing the final clusters
df_final_somK.drop(columns='BMU').groupby('labels').mean()

In [ ]:
get_r2(df_final_somK)

<a class="anchor" id="12.3.2">

### 12.3.2 Hierarchical Clustering on top of SOM
    
</a>


In [ ]:
df_somH = df_som.copy()

In [ ]:
methods = ["ward", "complete", "average", "single"]
distance = 'euclidean'
draw_r2_plot_hierarchical(df_somH,methods,distance)

we still use ward as is the best for higher clusters after 7 and is quite similar to the others <7

In [ ]:
# setting distance_threshold=0 and n_clusters=None ensures we compute the full tree
linkage = 'ward'
distance = 'euclidean'
hclust = AgglomerativeClustering(linkage=linkage, affinity=distance, distance_threshold=0, n_clusters=None)
hclust.fit_predict(df_somH[metric_features])

In [ ]:
draw_dendrogram(15,hclust)

select 4 clusters

In [ ]:
# Perform Hierarchical clustering on top of the 2500 untis (sm.get_node_vectors() output)
hierclust = AgglomerativeClustering(n_clusters=4, linkage='ward')
nodeclus_labels = hierclust.fit_predict(sm.codebook.matrix)
sm.cluster_labels = nodeclus_labels  # setting the cluster labels of sompy

hits  = HitMapView(12, 12,"Clustering",text_size=10)
hits.show(sm, anotate=True, onlyzeros=False, labelsize=7, cmap="Pastel1")

plt.show()

In [ ]:
# Check the nodes and and respective clusters
nodes_H = sm.codebook.matrix

df_nodes_H = pd.DataFrame(nodes_H, columns=metric_features)
df_nodes_H['labels'] = nodeclus_labels
df_nodes_H 

In [ ]:
# Obtaining SOM's BMUs labels
bmus_map_H = sm.find_bmu(df_somH[metric_features])[0]  # get bmus for each observation in df

df_bmus_H = pd.DataFrame(
    np.concatenate((df_somH, np.expand_dims(bmus_map_H,1)), axis=1),
    index=df_somH.index, columns=np.append(df_somH.columns,"BMU")
)
df_bmus_H

In [ ]:
# Get cluster labels for each observation
df_final_somH = df_bmus_H.merge(df_nodes_H['labels'], 'left', left_on="BMU", right_index=True)
df_final_somH

In [ ]:
# Characterizing the final clusters
df_final_somH.drop(columns='BMU').groupby('labels').mean()

In [ ]:
get_r2(df_final_somH)

### Final SOM Clustering solution

The r2 for kmeans on top of SOM (0.2332850987604802) is better then the Hierarchical Clustering on top of SOM units (0.20772534557034383)

<a class="anchor" id="12.4">

## 12.4. Birch Clustering
    
</a>


In [ ]:
def get_ss(df):
    """Computes the sum of squares for all variables given a dataset
    """
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable

def r2(df, labels):
    sst = get_ss(df)
    ssw = np.sum(df.groupby(labels).apply(get_ss))
    return 1 - ssw/sst
    
def get_r2_scores(df, clusterer, min_k=2, max_k=10):
    """
    Loop over different values of k. To be used with sklearn clusterers.
    """
    r2_clust = {}
    for n in range(min_k, max_k):
        clust = clone(clusterer).set_params(n_clusters=n)
        labels = clust.fit_predict(df)
        r2_clust[n] = r2(df, labels)
    return r2_clust

In [ ]:
df_birch = df_before_models.copy()

## Optimal Clusters

In [ ]:
birch_model = Birch()
r2_birch_score = {}
r2_birch_score['birch'] = get_r2_scores(df_birch,birch_model)

In [ ]:
r2_birch_score

In [ ]:
pd.DataFrame(r2_birch_score).plot.line(figsize=(10,7))

plt.title("Birch Clustering:\nR² plot for various number of clusters\n", fontsize=21)
plt.legend(title="Cluster method", title_fontsize=11)
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R² metric", fontsize=13)
plt.show()

## Implementing Birch Clustering

In [ ]:
# Final Cluster Solution
birch_model = Birch(n_clusters = 4,threshold = 0.4)
birch_labels = birch_model.fit_predict(df_birch[metric_features])
birch_labels

In [ ]:
# Characterizing the final clusters
df_final_birch = pd.concat((df_birch,pd.Series(birch_labels,name = 'labels')),axis=1)
df_final_birch.groupby('labels').mean()

In [ ]:
get_r2(df_final_birch)

<a class="anchor" id="12.5">

## 12.5. Clustering by Perspectives
    
</a>


In [ ]:
df_before_models[metric_features].columns

In [ ]:
services_features = [
    'PremMotor',
    'PremHousehold',
    'PremHealth',
    'PremLife',
    'PremWork',
    'TotalPremiums'
]

customer_features = [
    'CustMonVal', 
    'YearSal', 
    'Premiums%Salary'
]

df_services = df_before_models[services_features].copy()
df_customer = df_before_models[customer_features].copy()

In [ ]:
kmeans = KMeans(
    init='k-means++',
    n_init=20,
    random_state=42
)

hierarchical = AgglomerativeClustering(
    affinity='euclidean'
)

birch = Birch(
    n_clusters=2, threshold =0.1
)


spectral = SpectralClustering()

## Services Perspective

In [ ]:
# Obtaining the R² scores for each cluster solution on services variables
r2_scores_services = {}
r2_scores_services['kmeans'] = get_r2_scores(df_services, kmeans)

for linkage in ['complete', 'average', 'single', 'ward']:
    r2_scores_services[linkage] = get_r2_scores(
        df_services, hierarchical.set_params(linkage=linkage)
    )
    
r2_scores_services['birch'] = get_r2_scores(df_services,birch)
r2_scores_services['spectral'] = get_r2_scores(df_services,spectral)

pd.DataFrame(r2_scores_services)

In [ ]:
# Visualizing the R² scores for each cluster solution on services variables
pd.DataFrame(r2_scores_services).plot.line(figsize=(10,7))

plt.title("Services Variables:\nR² plot for various clustering methods\n", fontsize=21)
plt.legend(title="Cluster methods", title_fontsize=11)
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R² metric", fontsize=13)
plt.savefig('plot_services_r2')
plt.show()

## Customer Perspective

In [ ]:
# Obtaining the R² scores for each cluster solution on customer variables
r2_scores_customer = {}
r2_scores_customer['kmeans'] = get_r2_scores(df_customer, kmeans)

for linkage in ['complete', 'average', 'single', 'ward']:
    r2_scores_customer[linkage] = get_r2_scores(
        df_customer, hierarchical.set_params(linkage=linkage)
    )
    
r2_scores_customer['birch'] = get_r2_scores(df_customer,birch)
r2_scores_customer['spectral'] = get_r2_scores(df_customer,spectral)

pd.DataFrame(r2_scores_customer)

In [ ]:
# Visualizing the R² scores for each cluster solution on customer variables
pd.DataFrame(r2_scores_customer).plot.line(figsize=(10,7))

plt.title("Customer Variables:\nR² plot for various clustering methods\n", fontsize=21)
plt.legend(title="Cluster methods", title_fontsize=11)
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R² metric", fontsize=13)
plt.show()

<a class="anchor" id="12.5.1">

### 12.5.1 K Means Clustering
    
</a>


### Confirming optimal number of clusters

In [ ]:
r_clusters = range(2, 10)

### Services Perspective

In [ ]:
inertia = []
for n_clus in r_clusters:  # iterate over desired ncluster range
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=15, random_state=1)
    kmclust.fit(df_services.copy())
    inertia.append(kmclust.inertia_)  # save the inertia of the given cluster solution

In [ ]:
draw_inertia_plot(r_clusters,inertia)

This plot shows us either 3 or 4 clusters, more inclined to choose 3 clusters

### Silhouette Scores

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

# Storing average silhouette metric
avg_silhouette = []
for nclus in range(2,10):
    # Skip nclus == 1
    if nclus == 1:
        continue
    
    # Create a figure
    fig = plt.figure(figsize=(13, 7))

    # Initialize the KMeans object with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    kmclust = KMeans(n_clusters=nclus, init='k-means++', n_init=15, random_state=1)
    cluster_labels = kmclust.fit_predict(df_services)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters
    silhouette_avg = silhouette_score(df_services, cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df_services, cluster_labels)

    y_lower = 10
    for i in range(nclus):
        # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        # Get y_upper to demarcate silhouette y range size
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        # Filling the silhouette
        color = cm.nipy_spectral(float(i) / nclus)
        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    plt.title("The silhouette plot for the various clusters.")
    plt.xlabel("The silhouette coefficient values")
    plt.ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    plt.axvline(x=silhouette_avg, color="red", linestyle="--")
    
    # The silhouette coefficient can range from -1, 1
    xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
    plt.xlim([xmin, xmax])
    
    # The (nclus+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    plt.ylim([0, len(df_services) + (nclus + 1) * 10])

    plt.yticks([])  # Clear the yaxis labels / ticks
    plt.xticks(np.arange(xmin, xmax, 0.1))

### Average Silhouette Plot

In [ ]:
draw_average_silhouette(range(2,10),avg_silhouette)

For the k means method, 3 clusters is the overall optimal number of clusters for the services perspective

### Customer Perspective

In [ ]:
inertia = []
for n_clus in range_clusters: 
    kmclust_cust = KMeans(n_clusters=n_clus, init='k-means++', n_init=15, random_state=1)
    kmclust_cust.fit(df_customer.copy())
    inertia.append(kmclust_cust.inertia_) 

In [ ]:
draw_inertia_plot(range_clusters,inertia)

### Silhouette Scores

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py
avg_silhouette = []
for nclus in range(1,11):
    if nclus == 1:
        continue
    fig = plt.figure(figsize=(13, 7))
    kmclust = KMeans(n_clusters=nclus, init='k-means++', n_init=15, random_state=1)
    cluster_labels = kmclust.fit_predict(df_customer)
    silhouette_avg = silhouette_score(df_customer, cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")
    sample_silhouette_values = silhouette_samples(df_customer, cluster_labels)
    y_lower = 10
    for i in range(nclus):
    
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        color = cm.nipy_spectral(float(i) / nclus)
        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10  

    plt.title("The silhouette plot for the various clusters.")
    plt.xlabel("The silhouette coefficient values")
    plt.ylabel("Cluster label")

    plt.axvline(x=silhouette_avg, color="red", linestyle="--")

    xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
    plt.xlim([xmin, xmax])
    plt.ylim([0, len(df_customer) + (nclus + 1) * 10])

    plt.yticks([]) 
    plt.xticks(np.arange(xmin, xmax, 0.1))

### Average Silhouette Plot

In [ ]:
draw_average_silhouette(range(2,11),avg_silhouette)

Customer perspective with k means clustering has an optimal amount of 4 clusters

## Final K-Means Solution

In [ ]:
df_kmeans_services = df_services.copy()

In [ ]:
kmclust_services = KMeans(
    n_clusters=3,
    init='k-means++',
    n_init=20,
    random_state=42
)
km_labels_services = kmclust_services.fit_predict(df_kmeans_services)


In [ ]:
df_kmeans_customer = df_customer.copy()

In [ ]:
kmclust_customer = KMeans(
    n_clusters=4,
    init='k-means++',
    n_init=20,
    random_state=42
)
km_labels_customer = kmclust_customer.fit_predict(df_kmeans_customer)

In [ ]:
df_merging = df_before_models.copy()

In [ ]:
df_merging['services_labels'] = km_labels_services
df_merging['customer_labels'] = km_labels_customer

## Creating contingency table

In [ ]:
df_merging.groupby(['customer_labels', 'services_labels'])\
    .size()\
    .to_frame()\
    .reset_index()\
    .pivot('customer_labels', 'services_labels', 0)

<a class="anchor" id="12.5.2">

### 12.5.2 Merging Perspectives
    
</a>


### Manual Merging

In [ ]:
to_merge = [(0,1),(1,1),(2,1),(3,0)] #the 4 cluster we are going to merge

df_centroids = df_merging.groupby(['customer_labels', 'services_labels'])\
    [metric_features].mean()

euclidean = pairwise_distances(df_centroids)
df_dists = pd.DataFrame(
    euclidean, columns=df_centroids.index, index=df_centroids.index
)

source_target = {}
for clus in to_merge:
    if clus not in source_target.values():
        source_target[clus] = df_dists.loc[clus].sort_values().index[1]

source_target

In [ ]:
df_ = df_merging.copy()

for source, target in source_target.items():
    mask = (df_['customer_labels']==source[0]) & (df_['services_labels']==source[1])
    df_.loc[mask, 'customer_labels'] = target[0]
    df_.loc[mask, 'services_labels'] = target[1]

# New contigency table
df_.groupby(['customer_labels', 'services_labels'])\
    .size()\
    .to_frame()\
    .reset_index()\
    .pivot('customer_labels', 'services_labels', 0)

In [ ]:
to_merge = [(1,0),(2,2),(1,2)] #the 3 cluster we are going to merge

df_second_merge = df_.copy()

df_centroids = df_second_merge.groupby(['customer_labels', 'services_labels'])\
    [metric_features].mean()

euclidean = pairwise_distances(df_centroids)
df_dists = pd.DataFrame(
    euclidean, columns=df_centroids.index, index=df_centroids.index
)

source_target = {}
for clus in to_merge:
    if clus not in source_target.values():
        source_target[clus] = df_dists.loc[clus].sort_values().index[1]

source_target

In [ ]:
for source, target in source_target.items():
    mask = (df_second_merge['customer_labels']==source[0]) & (df_second_merge['services_labels']==source[1])
    df_second_merge.loc[mask, 'customer_labels'] = target[0]
    df_second_merge.loc[mask, 'services_labels'] = target[1]

# New contigency table
df_second_merge.groupby(['customer_labels', 'services_labels'])\
    .size()\
    .to_frame()\
    .reset_index()\
    .pivot('customer_labels', 'services_labels', 0)

In [ ]:
#Re-name the cluster with the following order
cluster_mapper_manual = {
 (0, 0): 1,
 (0, 2): 2,
 (1, 0): 3,
 (1, 1): 4,
 (1, 2): 5,
 (2, 0): 6,
 (2, 2): 7,
 (3, 1): 8,
 (3, 2): 9
}

In [ ]:
df_mapped = df_.copy()
df_mapped['merged_labels'] = df_mapped.apply(
    lambda row: cluster_mapper_manual[
        (row['customer_labels'], row['services_labels'])
    ], axis=1
)

In [ ]:
#Re-name the cluster with the following order
cluster_mapper_manual = {
 (0, 0): 1,
 (0, 2): 2,
 (1, 1): 3,
 (2, 0): 4,
 (3, 1): 5,
 (3, 2): 6
}

In [ ]:
df_second_mapped = df_second_merge.copy()
df_second_mapped['merged_labels'] = df_second_mapped.apply(
    lambda row: cluster_mapper_manual[
        (row['customer_labels'], row['services_labels'])
    ], axis=1
)

### Merging using Hierarchical clustering

In [ ]:
df_centroids = df_.groupby(['services_labels', 'customer_labels'])\
    [metric_features].mean()
df_centroids

In [ ]:
hclust = AgglomerativeClustering(
    linkage='ward', 
    affinity='euclidean', 
    distance_threshold=0, 
    n_clusters=None
)
hclust_labels = hclust.fit_predict(df_centroids)

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

# create the counts of samples under each node (number of points being merged)
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)

# hclust.children_ contains the observation ids that are being merged together
# At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
for i, merge in enumerate(hclust.children_):
    # track the number of observations in the current cluster being formed
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            # If this is True, then we are merging an observation
            current_count += 1  # leaf node
        else:
            # Otherwise, we are merging a previously formed cluster
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

# the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
# the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
# the counts indicate the number of points being merged (dendrogram's x-axis)
linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)

# Plot the corresponding dendrogram
sns.set()
fig = plt.figure(figsize=(11,5))
# The Dendrogram parameters need to be tuned
y_threshold = 0.3
dendrogram(linkage_matrix, truncate_mode='level', labels=df_centroids.index, p=5, color_threshold=y_threshold, above_threshold_color='k')
plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram', fontsize=21)
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'Euclidean Distance', fontsize=13)
plt.show()

In [ ]:
# Re-running the Hierarchical clustering based on the correct number of clusters
hclust = AgglomerativeClustering(
    linkage='ward', 
    affinity='euclidean', 
    n_clusters=7
)
hclust_labels = hclust.fit_predict(df_centroids)
df_centroids['hclust_labels'] = hclust_labels

df_centroids  # centroid's cluster labels

In [ ]:
# Mapper between concatenated clusters and hierarchical clusters
cluster_mapper = df_centroids['hclust_labels'].to_dict()

df_second_merge = df_.copy()

# Mapping the hierarchical clusters on the centroids to the observations
df_second_merge['merged_labels'] = df_second_merge.apply(
    lambda row: cluster_mapper[
        (row['services_labels'], row['customer_labels'])
    ], axis=1
)

# Merged cluster centroids
df_second_merge.groupby('merged_labels').mean()[metric_features]

In [ ]:
df_counts = df_second_merge.groupby('merged_labels')\
    .size()\
    .to_frame()

df_counts = df_counts\
    .rename({v:k for k, v in cluster_mapper.items()})\
    .reset_index()

df_counts['services_labels'] = df_counts['merged_labels'].apply(lambda x: x[0])
df_counts['customer_labels'] = df_counts['merged_labels'].apply(lambda x: x[1])
df_counts.pivot('services_labels', 'customer_labels', 0)

<a class="anchor" id="13">

# 13. Cluster Visualization
    
</a>


<a class="anchor" id="13.1">

## 13.1 Using t-SNE
    
</a>


### Clustering solution using initial merge + hierarchical clustering

In [ ]:
df_tsne = df_second_merge.copy()

In [ ]:
two_dim = TSNE(random_state=42).fit_transform(df_tsne[metric_features])

In [ ]:
pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=df_tsne['merged_labels'], colormap='tab10', figsize=(15,10))
plt.show()

### Clustering Solution using only initial manual merging

In [ ]:
df_tsne_manual = df_mapped.copy()

In [ ]:
two_dim = TSNE(random_state=42).fit_transform(df_tsne_manual[metric_features])

In [ ]:
pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=df_tsne_manual['merged_labels'], colormap='tab10', figsize=(15,10))
plt.show()

### Clustering Solution using second manual merging

In [ ]:
df_tsne_manual = df_second_mapped.copy()

In [ ]:
two_dim = TSNE(random_state=42).fit_transform(df_tsne_manual[metric_features])

In [ ]:
pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=df_tsne_manual['merged_labels'], colormap='tab10', figsize=(15,10))
plt.show()

<a class="anchor" id="13.2">

## 13.2 Using Polar Line Plot
    
</a>


In [ ]:
df_visualization = df_second_mapped.copy()

In [ ]:
# Adapted from: https://towardsdatascience.com/clustering-with-more-than-two-features-try-this-to-explain-your-findings-b053007d680a
clusters = df_visualization[metric_features].copy()
clusters['label']= df_visualization['merged_labels']
polar=clusters.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=500,width=700)
fig4.show()

<a class="anchor" id="13.3">

## 13.3 Using Cluster Profiles
    
</a>


In [ ]:
df_cluster_analysis = df_visualization.copy()

In [ ]:
def cluster_profiles(df, label_columns, figsize, compar_titles=None):
    """
    Pass df with labels columns of one or multiple clustering labels. 
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)
        
    sns.set()
    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2, figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):
        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)
        
        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]
        
        # Setting Data
        pd.plotting.parallel_coordinates(centroids, label, color=sns.color_palette(), ax=ax[0])
        sns.barplot(x=label, y="counts", data=counts, ax=ax[1])

        #Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(s=titl, xy=(0.95,1.1), xycoords='axes fraction', fontsize=13, fontweight = 'heavy') 
        ax[0].legend(handles, cluster_labels) # Adaptable to number of clusters
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)), fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=-20)
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)), fontsize=13)
    
    plt.subplots_adjust(hspace=0.4, top=0.90)
    plt.suptitle("Cluster Simple Profilling", fontsize=23)
    plt.show()

In [ ]:
# Profilling each cluster (product, behavior, merged)
cluster_profiles(
    df = df_cluster_analysis[metric_features + ['customer_labels', 'services_labels', 'merged_labels']], 
    label_columns = ['customer_labels', 'services_labels', 'merged_labels'], 
    figsize = (28, 13), 
    compar_titles = ["Customer clustering", "Services clustering", "Merged clusters"]
)

# Feature Importance and outliers reclassification

## R2

In [ ]:
def get_ss_variables(df):
    """Get the SS for each variable
    """
    ss_vars = df.var() * (df.count() - 1)
    return ss_vars

def r2_variables(df, labels):
    """Get the R² for each variable
    """
    sst_vars = get_ss_variables(df)
    ssw_vars = np.sum(df.groupby(labels).apply(get_ss_variables))
    return 1 - ssw_vars/sst_vars

In [ ]:
# We are essentially decomposing the R² into the R² for each variable
r2_variables(df_cluster_analysis[metric_features + ['merged_labels']], 'merged_labels').drop('merged_labels')

PremiumMotor, PremiumLife and YearSal are the best variables to distinguish the different clusters beacuase they have the highest R square

## Decision Tree

In [ ]:
# Preparing the data
X = df_cluster_analysis.drop(columns=['customer_labels','services_labels','merged_labels'])
y = df_cluster_analysis.merged_labels

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fitting the decision tree
dt = DecisionTreeClassifier(random_state=42, max_depth=3)
dt.fit(X_train, y_train)
print("It is estimated that in average, we are able to predict {0:.2f}% of the customers correctly".format(dt.score(X_test, y_test)*100))

In [ ]:
# Assessing feature importance
pd.Series(dt.feature_importances_, index=X_train.columns)

In [ ]:
# Predicting the cluster labels of the outliers
df_outliers_dbscan['merged_labels'] = dt.predict(df_outliers_dbscan)


In [ ]:
df_outliers_dbscan['merged_labels'].value_counts()

In [ ]:
all_data = pd.concat([df_outliers_dbscan,df_cluster_analysis]) 

In [ ]:
two_dim_outliers = TSNE(random_state=42).fit_transform(df_outliers_dbscan[metric_features])

In [ ]:
pd.DataFrame(two_dim_outliers).plot.scatter(x=0, y=1, c=df_outliers_dbscan['merged_labels'], colormap='tab10', figsize=(15,10))
plt.show()

In [ ]:
two_dim_outliers = TSNE(random_state=42).fit_transform(all_data[metric_features])

In [ ]:
pd.DataFrame(two_dim_outliers).plot.scatter(x=0, y=1, c=all_data['merged_labels'], colormap='tab10', figsize=(15,10))
plt.show()